In [1]:
import pandas as pd

In [2]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

In [3]:
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']]

# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.head()

,name,code
0,BYC,001460
1,CJ CGV,079160
2,DB손해보험,005830
3,DGB금융지주,139130
4,DSR제강,069730


In [4]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    
    print("요청 URL = {}".format(url)) 
    return url

# 신라젠의 일자데이터 url 가져오기 
item_name='신라젠' 
url = get_url(item_name, code_df)

# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame()

# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 21): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 

# 상위 5개 데이터 확인하기 
df.head()

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
0,2018.05.24,80100,5700,73600,80500,73500,2367206
1,2018.05.23,74400,2500,77500,78100,74300,1255645
2,2018.05.21,76900,200,77400,78700,76800,903245
3,2018.05.18,76700,0,77400,79400,76300,1547616
4,2018.05.17,76700,1700,76000,78500,73700,2125813


In [5]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 
    
# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
df.head()

,date,close,diff,open,high,low,volume
199,2017-07-27,23900,1000,23100,24250,22950,1669657
198,2017-07-28,24100,200,24250,24500,23500,1607694
197,2017-07-31,24800,700,23850,24800,23400,1349443
196,2017-08-01,25700,900,24850,26350,24700,3246676
195,2017-08-02,24100,1600,25900,26200,23800,3115525


In [6]:
def get_macd(df, short=12, long=26, t=9): 

    # 입력받은 값이 dataframe이라는 것을 정의해줌 
    df = pd.DataFrame(df) 
    
    # MACD 관련 수식 
    ma_12 = df.close.ewm(span=12).mean() # 단기(12) EMA(지수이동평균) 
    ma_26 = df.close.ewm(span=26).mean() # 장기(26) EMA 
    macd = ma_12 - ma_26 # MACD 
    macds = macd.ewm(span=9).mean() # Signal 
    macdo = macd - macds # Oscillator 
    
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna() 
    
    return df

In [7]:
df = get_macd(df) 

# 상위 5개 데이터 확인 
df.head()

,date,close,diff,open,high,low,volume,macd,macds,macdo
199,2017-07-27,23900,1000,23100,24250,22950,1669657,0.000000,0.000000,0.000000
198,2017-07-28,24100,200,24250,24500,23500,1607694,4.487179,2.492877,1.994302
197,2017-07-31,24800,700,23850,24800,23400,1349443,27.424515,12.710762,14.713754
196,2017-08-01,25700,900,24850,26350,24700,3246676,69.678917,32.008918,37.670000
195,2017-08-02,24100,1600,25900,26200,23800,3115525,27.504729,30.669023,-3.164295


In [8]:
from plotly import tools 
import plotly.offline as offline 
import plotly.graph_objs as go

In [10]:
offline.init_notebook_mode(connected=True) 

macd = go.Scatter( 
    x=df.date, 
    y=df['macd'], 
    name="MACD") 

signal = go.Scatter( 
    x=df.date, 
    y=df['macds'], 
    name="Signal") 

oscillator = go.Bar( x=df.date, 
    y=df['macdo'], 
    name="oscillator") 

trade_volume = go.Bar( x=df.date, 
    y=df['volume'], 
    name="volume") 

data = [macd, signal, oscillator] 

# data = [celltrion] 
layout = go.Layout(title='{} MACD 그래프'.format(item_name)) 

fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True) 

for trace in data: 
    fig.append_trace(trace, 1,1) 
    
fig.append_trace(trade_volume, 2,1) 
# fig = go.Figure(data=data, layout=layout) 

offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

